# 計算ノードの追加

このnotebookでは、OCS OpenHPC-v2テンプレートで作成されたSlurmクラスタに計算ノードを追加する手順について説明します。

## 前提

* このnotebookで動作確認しているのは、mdx VM上に構築されたOpenHPC環境のみです。他のクラウドプロバイダ上に構築された環境に適用する場合には変更が必要かもしれません。
* 正常にジョブ実行ができる状態となっている状態を前提とします。構築途中であったり不具合のある状態の環境は対象としません。
* 既存の計算ノードのIPアドレスは連続した状態で構築されているものとします。また、このnotebookで新規に追加される計算ノードのIPアドレスは、既存の計算ノードのIPアドレスの4オクテット目に1加算されたものとなります。

## 準備

### UnitGroup名の指定

構築環境の UnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために group_vars ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/*.yml | sed -e 's/^group_vars\///' -e 's/\.yml//' | sort

UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'OpenHPC'

ugroup_name = 

### VCCアクセストークンの入力

VCノードを起動するにはVC Controller(VCC)にアクセスして、操作を行う必要があります。VCCにアクセスするために必要となるアクセストークンをここで入力します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
config vc failed: http_status(403)
2021/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

> `unfreeze`するにはNotebookのツールバーにある`unfreeze below in section`ボタンなどを利用してください。

### group_varsの読み込み

次のセルを実行すると「010-パラメータの設定.ipynb」で指定したパラメータを読み込みます。読み込むパラメータの値は、UnitGroup名に指定した 値に対応するものになります。UnitGroup名の指定が誤っていると意図したパラメータが読み込めないので注意してください。

In [ ]:
%run scripts/group.py

gvars = load_group_vars(ugroup_name)

group_varsの内容を確認しておきます。

In [ ]:
!cat group_vars/{ugroup_name}.yml

## 追加するノードのパラメータ設定

計算ノード追加の準備として、変数などの設定を実施します。

追加するノード数を`nodes_to_add`に設定します。

In [ ]:
nodes_to_add = 1

以下のセルで計算ノードのホスト名からホスト名部分のプレフィックス部分を推定し、`c_hostname_prefix`に設定します。設定内容が表示されます。

In [ ]:
cn_present = gvars['compute_etc_hosts']
cn_max = max(cn_present.values(), key=len)
for i in range(len(cn_max) - 2):
    c_hostname_prefix = cn_max[0:-(i+1)]
    for n in list(cn_present.values()):
        matched = n.startswith(c_hostname_prefix)
        if not matched:
            break
    if matched:
        break

c_hostname_prefix

表示されたプレフィックスが正しくない場合は、以下のセルのコメントを外して、`c_hostname_prefix`に正しいプレフィックスを設定してください。

In [ ]:
# c_hostname_prefix = ''

以下のセルで、追加する計算ノードに割り当てる先頭のIPアドレスを推定します。

In [ ]:
import ipaddress

c_addr_base = max(map(ipaddress.ip_address,
                      gvars['compute_etc_hosts'].keys())) + 1
str(c_addr_base)

推定されたアドレスが正しくない場合には、以下のセルのコメントを外して、`c_addr_base`に正しいIPアドレスを設定してください。

In [ ]:
# c_addr_base = ipaddress.ip_address('192.168.1.30')

今後の作業のために、追加する計算ノードのためのデータ構造を作成します。

In [ ]:
c_add_etc_hosts = {
        f'{c_addr_base + i}':
            c_hostname_prefix + str(gvars['compute_nodes'] + i + 1)
        for i in range(nodes_to_add)
    }

新規に追加する計算ノードのIPアドレスが適切かどうかチェックします。

In [ ]:
%run scripts/utils.py

check_parameters(
    _params=dict(
        vcp=vcp,
        vc_provider=gvars['vc_provider'],
        compute_nodes=nodes_to_add,
    ),
    compute_etc_hosts=c_add_etc_hosts,
)

## mdx VMの起動

計算ノードとなるmdx VMを起動し、VCノードとして使用できる状態にします。VM起動時点でVCノードとして使用できる環境では、この章をスキップすればこのnotebookを使用できるよう意図しています(検証はしていません)。

### mdx操作の準備

以下のセルを実行してmdx REST API認証トークンを入力します。

In [ ]:
from getpass import getpass
mdx_token = getpass("mdx API token")

mdx REST APIエンドポイントにIPv6で接続しようとすると到達不可となる場合があるため、以下のセルを実行してIPv4での接続を強制します。

In [ ]:
def use_ipv4_only():
    import socket
    old_getaddrinfo = socket.getaddrinfo
    def new_getaddrinfo(*args, **kwargs):
        responses = old_getaddrinfo(*args, **kwargs)
        return [response
                for response in responses
                if response[0] == socket.AF_INET]
    socket.getaddrinfo = new_getaddrinfo

use_ipv4_only()

VCP SDK mdx用プラグインモジュールを読み込みます。

In [ ]:
from common import logsetting
from vcpsdk.plugins.mdx_ext import MdxResourceExt
mdx = MdxResourceExt(mdx_token)

### 計算ノード用VMパラメータの設定

以下のセルを実行することで、計算ノードのVMのパラメータを作成します。作成するVMは既存の計算ノードと同じパラメータ設定となります。

In [ ]:
%run scripts/mdx_ops.py

# disk_size
if 'compute_root_size' in gvars:
    disk_size = gvars['compute_root_size']
else:
    disk_size = 40

# shared_key読み込み
import os
with open(os.path.expanduser(gvars['ssh_public_key_path'])) as f:
    shared_key = f.read()

c_spec = mdx_get_vm_spec(
    mdx,
    gvars['mdx_compute_pack_num'],
    gvars['compute_use_gpu'],
    disk_size,
    gvars['mdx_segment_id'],
    shared_key
)

2023-01-31のmdx REST APIの仕様変更により、VMのspecに`service_level`を指定しないとVMデプロイのリクエストがエラーとなるようになりました。このため、`service_level`を`guarantee`に指定して、明示的に起動保証VMとしてデプロイするよう指定します。

In [ ]:
from vcpsdk.plugins.mdx_ext import MDX_VM_SPEC_SCHEMA

if not 'service_level' in MDX_VM_SPEC_SCHEMA['properties']:
    MDX_VM_SPEC_SCHEMA['properties'].update(
        {'service_level': {'type': 'string'}}
    )

c_spec.update({'service_level': 'guarantee'})

VM内のユーザの初期パスワードを`initial_passwd`に設定します。

In [ ]:
initial_passwd = 'mdx additional cnode passwd'

### VMデプロイ

追加する計算ノードをmdx上にデプロイし、IPアドレスの変更を実施します。

VMのデプロイと設定に数分程度要するため、実行中のセルの経過時間を表示するライブラリjupyter-autotime機能を有効化します。

In [ ]:
!pip install --no-deps jupyter-autotime
%load_ext autotime

追加する計算ノードをデプロイし、外部からアクセスできるようになるまで待ちます。この処理には数分程度要します。

In [ ]:
%run scripts/mdx_ops.py

mdx_deploy_vms(mdx, list(c_add_etc_hosts.values()), c_spec,
               project=gvars['mdx_project_name'], verbose=True)

VMに対し初期パスワードを設定します。

In [ ]:
%run scripts/mdx_ops.py

mdx_set_init_passwd(mdx, c_add_etc_hosts.values(),
                   gvars['ssh_private_key_path'],
                   initial_passwd)

IPアドレスを変更します。

In [ ]:
%run scripts/mdx_ops.py

mdx_change_ipaddrs(mdx, c_add_etc_hosts,
                   os.path.expanduser(gvars['ssh_private_key_path']),
                   verbose=True)

### VCノード向け設定

mdx VMに対し、VCノードとして使用できるよう設定します。

In [ ]:
%run scripts/mdx_ops.py

vcppubkey = vcp.get_publickey()
mdx_init_vcp(list(c_add_etc_hosts.keys()),
             os.path.expanduser(gvars['ssh_private_key_path']),
            vcppubkey)

## Slurmへの組み込み

起動したmdx VMをVC Unitに組み込み、必要な設定更新をした後でSlurmに組み込みます。

### VC unitへの組み込み

起動したmdx VMを計算ノードのUnitに組み込むことで、OpenHPCを導入します。

UnitGroupの状態を表示し、組み込み先のUnitを確認します。

In [ ]:
ug = vcp.get_ugroup(gvars['ugroup_name'])
ug.df_units()

mdx VMを計算ノードのUnitに組み込みます。

In [ ]:
unit = ug.get_unit('compute')
unit.add_nodes(num_add_nodes=nodes_to_add,
               ip_addresses=list(c_add_etc_hosts.keys()))

### ansibleの設定更新

追加したVCノードをansibleでの操作対象とするため、ansibleの設定を更新します。

SSHのknown_hostsファイルに追加したノードのエントリを追加します。

In [ ]:
from time import sleep

def check_update_known_hosts(ipaddr):
    # VCノード起動直後だと sshd サービスが開始されておらずに known_hosts が更新されない場合がある
    # ssh-keyscan が値を取得できるまで何度かリトライする
    for x in range(10):
        out = ! echo $(ssh-keyscan {ipaddr} 2> /dev/null | wc -l)
        update_lines = int(out[0])
        if update_lines > 0:
            break
        sleep(1)
    else:
        raise RuntimeError("ERROR: timeout!")    

for addr in list(c_add_etc_hosts.keys()):
    !ssh-keygen -R {addr}
    check_update_known_hosts(addr)
    !ssh-keyscan -H {addr} >> ~/.ssh/known_hosts

ansibleの`inventory.yml`を更新します。

In [ ]:
import yaml

inventory = {
    'all': {
        'children': {
            ug.name: {
                'children': dict([
                    (
                        f'{ug.name}_{unit.name}',
                        {
                            'hosts': dict([
                                (ip, {})
                                for ip in unit.find_ip_addresses()
                            ])
                        }
                    )
                    for unit in ug.find_units()]),
                'vars': {
                    'ansible_user': 'vcp',
                    'ansible_ssh_private_key_file': gvars['ssh_private_key_path'],
                }
            }
        }
    }
}

with open('inventory.yml', 'w') as f:
    yaml.safe_dump(inventory, f, default_flow_style=False)
    
!cat inventory.yml

追加したノードにansibleでアクセスできるか確認します。

In [ ]:
!ansible {ug.name} -m ping

### Slurmクラスタノードの設定ファイル更新

追加したVCノードが正しく既存のSlurmクラスタに組み込めるよう、各ノードの設定ファイルを更新します。具体的には以下のファイルを更新します。
* `/etc/hosts` (全VCノード)
* ホスト名(追加VCノード)
* アカウント関連ファイル(追加VCノード)
* `/etc/slurm/slurm.conf` (全VCノード)

`/etc/hosts`を更新します。

In [ ]:
for addr, host in c_add_etc_hosts.items():
    !ansible {ug.name} -m lineinfile -b -a \
        "path=/etc/hosts line='{addr}\t{host}' \
        state=present unsafe_writes=true"

追加VCノードのホスト名を設定します。OpenHPC-v2テンプレートで使用しているコンテナでは、VcpSDKの`Unit.add_node()`でVCノードを追加した場合にホスト名が正しく設定されないため、別途設定します。

In [ ]:
for addr in c_add_etc_hosts.keys():
    !ansible {addr} -m shell -b -a \
        "PRIVATE_IP={addr} sh /etc/vcp/rc.d/00-hostname.sh && \
            hostnamectl set-hostname --transient `cat /etc/hostname`"

追加VCノードのユーザアカウント関連のファイルを、既存の計算ノードと一致させます。

In [ ]:
filemodes = {
    '/etc/passwd': '0644',
    '/etc/group': '0644',
    '/etc/shadow': '0000'
}
files = list(filemodes.keys())
srchost = list(gvars['compute_etc_hosts'].keys())[0]
tmpd = 'tmp.account'

!mkdir -p {tmpd}
for f in files:
    !ansible {srchost} -m fetch -b -a \
        "dest={tmpd} src={f}"
for addr in c_add_etc_hosts.keys():
    for f in files:
        !ansible {addr} -m copy -b -a \
            "src={tmpd}/{srchost}/{f} dest={f} mode={filemodes[f]}"
!rm -rf {tmpd}

`/etc/slurm/slurm.conf`を更新します。まず、現状の`slurm.conf`のベースとなっているgroup_varsの設定を確認します。

In [ ]:
import json
print(json.dumps(gvars['slurm_conf'], indent=2))

`NodeName`に、追加VCノードが含まれるように更新します。以下の処理では正しく更新できない場合には、別途セルを編集の上設定してください。

In [ ]:
from copy import deepcopy

compute_nodes_new = gvars['compute_nodes'] + nodes_to_add
gvars_new = deepcopy(gvars)
gvars_new['slurm_conf'].update(
    {'NodeName': f'{c_hostname_prefix}[1-{compute_nodes_new}]'}
)

print(json.dumps(gvars_new['slurm_conf'], indent=2))

各ノードの`slurm.conf`を更新します。更新に先立って必要なスクリプトを各ノードにコピーしています。

In [ ]:
# slurmfuncs.shを配布
!ansible {ug.name} -m copy -b -a \
    'src=scripts/slurmfuncs.sh dest=/etc/vcp/'

# 配布したスクリプトを使用してslurm.confを更新
%run scripts/utils.py
gvars_new['compute_etc_hosts'].update(c_add_etc_hosts)
slurm_params = spec_env_slurm_conf(gvars_new)
!ansible {ug.name} -m file -b -a \
    'path=/var/lib/vcp/.20-slurm state=absent'
!ansible {ug.name} -m shell -b -a \
    ". /etc/vcp/slurmfuncs.sh; \
    SLURM_NODE_PARAMS={slurm_params} setup_slurm_conf && \
    MASTER_HOSTNAME={gvars_new['master_hostname']} setup_control_machine"

### Slurmへの追加VCノードの組み込み

`slurmctld`, `slurmd`それぞれに対して設定更新の通知をすることで、追加したVCノードがジョブのスケジュール対象となるようにします。

`slurmctld`を再起動します。

In [ ]:
!ansible {ug.name}_master -m systemd -b -a \
    'name=slurmctld state=restarted'

Slurmクラスタの再構成を指示します。

In [ ]:
!ansible {ug.name}_master -b -a \
    'scontrol reconfigure'

追加ノードの`slurmd`を再起動します。

In [ ]:
for addr in c_add_etc_hosts.keys():
    !ansible {addr} -m systemd -b -a \
        'name=slurmd state=restarted'

Slurmの状態を確認し、追加ノードが正しく追加されていることを確認します。

In [ ]:
!ansible {ug.name}_master -b -a 'sinfo'

追加ノードも必要となるジョブを実行して、実際に追加ノードが組み込まれていることを確認します。

In [ ]:
!ansible {ug.name}_master -a 'srun -l -N {compute_nodes_new} hostname'

## notebook環境側のデータ更新

後でさらにノードを追加したり、ノードを削除する場合に備えて、ノードを追加した状態の設定に合わせて、noetbook環境側のデータを更新します。

### group_varsの更新

ノードを追加した際に更新する必要がある、以下のパラメータを更新します。
* `slurm_conf`
* `compute_nodes`
* `compute_etc_hosts`

In [ ]:
%run scripts/group.py

update_group_vars(
    ugroup_name,
    slurm_conf=gvars_new['slurm_conf'],
    compute_nodes=compute_nodes_new,
    compute_etc_hosts=gvars_new['compute_etc_hosts']
)

group_varsの内容を確認します。

In [ ]:
!cat group_vars/{ugroup_name}.yml

## その他

このnotebookでは扱いませんが、計算ノードに対してパッケージの追加や設定ファイルの更新などを実施している場合は、別途追加した計算ノードにも同じ設定が必要です。